Generate analysis (applying aiohttp)

In [1]:
import asyncio
import aiohttp
import zstandard as zstd
import logging
logger = logging.getLogger(__name__)
EXTERNAL_API_URL = "https://jsonplaceholder.typicode.com/posts/1"

# response=requests.get(EXTERNAL_API_URL)
#         # response.raise_for_status()
#         # data=response.json()


async with aiohttp.ClientSession(auto_decompress=False) as session:
    try:
        async with session.get(EXTERNAL_API_URL, headers={"Accept-Encoding": "zstd"}) as resp:
            compressed_data = await resp.read()
            encoding = resp.headers.get("Content-Encoding","none")
            print("Content-Encoding",encoding)
            if encoding == "zstd":
                    dctx = zstd.ZstdDecompressor()
                    with dctx.stream_reader(compressed_data) as reader:
                        data = reader.read().decode("utf-8")
            else:
                    data = compressed_data.decode("utf-8")
            print("data :",data)
            
            logger.debug(f"External API response data: {data}")
            logger.info(f"Analysis generated  successfully")

    except Exception as e:
        
        logger.error(f"Error occurred while generating analysis: {e}")

Content-Encoding zstd
data : {
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}


In [2]:
data

'{\n  "userId": 1,\n  "id": 1,\n  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",\n  "body": "quia et suscipit\\nsuscipit recusandae consequuntur expedita et cum\\nreprehenderit molestiae ut ut quas totam\\nnostrum rerum est autem sunt rem eveniet architecto"\n}'

In [3]:
print(type(data))

<class 'str'>


In [4]:
import json
j_data=json.loads(data)

In [5]:
j_data['title']

'sunt aut facere repellat provident occaecati excepturi optio reprehenderit'

Using requests

In [6]:
import requests
url='https://jsonplaceholder.typicode.com/posts/1'
response = requests.get(url)
print(response.json()['title'])

sunt aut facere repellat provident occaecati excepturi optio reprehenderit


Using HTTPX

In [ ]:
#sending request with httpx
import httpx
async with httpx.AsyncClient() as client:
    response = await client.get(EXTERNAL_API_URL)
    response.raise_for_status()
    data = response.json()
    logger.debug(f"External API response data: {data}")        

In [8]:
data

{'userId': 1,
 'id': 1,
 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}

Improving the generate analysis function to generate using LLM

In [ ]:
EXTERNAL_API_URL = "http://localhost:3000/focus"

In [4]:
user_id=1

In [5]:
focus_id=1

Focus Service

In [6]:
from app.schemas.schemas import FocusOutput,FocusReport
from core.logger import logger
import requests

EXTERNAL_API_URL = "http://localhost:3000/focus"

class FocusService:
    def __init__(self):
        pass

    def get_focus_details(self,focus_id: int) -> FocusOutput:
        """Service class to get all the data related to specific focus."""
    
        #sending request to external API to get focus details
        try:
            response=requests.get(f"{EXTERNAL_API_URL}/{focus_id}")
            response.raise_for_status()
            data=response.json()
            logger.debug(f"External API response data for focus_id={focus_id}: {data}")
            # return FocusOutput(**data)
        
            return FocusOutput(
                focus_id=focus_id,
                focus_name=data["focus_name"],
                reports=[FocusReport(**report) for report in data.get("reports", [])]
            )
        except requests.exceptions.RequestException as e:
            logger.error(f"Error while fetching focus details from external API for focus_id={focus_id}: {e}")
            raise

In [8]:
focus_service = FocusService()
focus_details = focus_service.get_focus_details(focus_id)
print(focus_details)

focus_id=1 focus_name='Cybersecurity Threats in Critical Infrastructure' reports=[FocusReport(country='USA', description='Increased ransomware attacks on energy and transportation sectors; mitigation measures include enhanced threat intelligence sharing and mandatory reporting of incidents.', created_at='2025-10-28T18:00:00'), FocusReport(country='UK', description='Phishing and supply-chain attacks targeting government agencies and healthcare systems.', created_at='2025-10-27T14:00:00')]


In [10]:
context = focus_details.reports
context

[FocusReport(country='USA', description='Increased ransomware attacks on energy and transportation sectors; mitigation measures include enhanced threat intelligence sharing and mandatory reporting of incidents.', created_at='2025-10-28T18:00:00'),
 FocusReport(country='UK', description='Phishing and supply-chain attacks targeting government agencies and healthcare systems.', created_at='2025-10-27T14:00:00')]

In [14]:
type(context)

list

In [24]:
context_text = "\n\n".join(
    [
        f"Country: {r.country}\nDescription: {r.description}\nDate: {r.created_at}"
        for r in context
    ]
)


In [25]:
context_text

'Country: USA\nDescription: Increased ransomware attacks on energy and transportation sectors; mitigation measures include enhanced threat intelligence sharing and mandatory reporting of incidents.\nDate: 2025-10-28T18:00:00\n\nCountry: UK\nDescription: Phishing and supply-chain attacks targeting government agencies and healthcare systems.\nDate: 2025-10-27T14:00:00'

In [12]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

/home/lahirum/Documents/Mirada/RnD/FastApi/test2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#-------------------------Initialize LLM-------------------------------
def load_llm():
    """
    Loads and initializes the Ollama LLM model globally.
    """
    global llm

    # load ML model
    try:                             
        logger.info("Initializing Ollama model...")
        # global llm
        llm= ChatOllama(
                model="llama3.2", 
                think=False,
                options={
                    "num_predict": 4096,
                    "temperature": 0.6,
                    "top_p": 0.95,
                    "top_k": 20,
                    "repeat_penalty": 1.05,
                }    
            )
    
        logger.info(f"Ollama model initialized successfully.{llm}")
        # Verify initialization
        if llm is None:
            raise ValueError("Model initialization returned None — check Ollama server status or model name.")
        logger.info(f"Ollama model initialized successfully: {llm.model}")
        return llm

    except Exception as e:
        # Catch all exceptions to prevent startup crashes
        logger.error(f"Failed to initialize model.: {e}")




In [16]:
llm=load_llm()

2025-11-03 12:20:02,549 - core.logger - INFO - Initializing Ollama model...
2025-11-03 12:20:02,560 - core.logger - INFO - Ollama model initialized successfully.model='llama3.2'
2025-11-03 12:20:02,560 - core.logger - INFO - Ollama model initialized successfully: llama3.2


In [17]:
llm

ChatOllama(model='llama3.2')

In [21]:
from langchain_core.prompts import ChatPromptTemplate

# Define a chat prompt template composed of system and user messages
chat_template= ChatPromptTemplate.from_messages([
    {
        "role": "system",
        "content": """Using the given reports included in the context,
give a comprehensive analysis report""",
    },
    {
        "role": "user",
        "content": f"""Context:{context}""",
    },
])


In [26]:
prompt = chat_template.format(context=context_text)

In [27]:
response = llm.invoke(prompt)

# Output the generated analysis
print(response.content)

**Comprehensive Analysis Report**

**Executive Summary:**
The recent reports from the United States and the United Kingdom highlight a growing concern of cybersecurity threats in critical sectors. This analysis report provides an overview of the reported incidents, their impact on the economy and public health, and potential mitigation measures to prevent similar attacks in the future.

**Incident Analysis:**

### USA - Increased Ransomware Attacks on Energy and Transportation Sectors

- **Description:** The report from the United States reveals a significant increase in ransomware attacks targeting critical energy and transportation sectors. This has resulted in disruptions to essential services, including power grid management and public transportation systems.
- **Impact:** The incidents have not only affected the functioning of critical infrastructure but also posed risks to national security and economic stability.

### UK - Phishing and Supply-Chain Attacks Targeting Government A

In [29]:
for key in response:
    print(key)

('content', '**Comprehensive Analysis Report**\n\n**Executive Summary:**\nThe recent reports from the United States and the United Kingdom highlight a growing concern of cybersecurity threats in critical sectors. This analysis report provides an overview of the reported incidents, their impact on the economy and public health, and potential mitigation measures to prevent similar attacks in the future.\n\n**Incident Analysis:**\n\n### USA - Increased Ransomware Attacks on Energy and Transportation Sectors\n\n- **Description:** The report from the United States reveals a significant increase in ransomware attacks targeting critical energy and transportation sectors. This has resulted in disruptions to essential services, including power grid management and public transportation systems.\n- **Impact:** The incidents have not only affected the functioning of critical infrastructure but also posed risks to national security and economic stability.\n\n### UK - Phishing and Supply-Chain Attac

In [ ]:
import asyncio
import aiohttp
import zstandard as zstd
import logging
logger = logging.getLogger(__name__)
EXTERNAL_API_URL = "https://jsonplaceholder.typicode.com/posts/1"

# response=requests.get(EXTERNAL_API_URL)
#         # response.raise_for_status()
#         # data=response.json()


async with aiohttp.ClientSession(auto_decompress=False) as session:
    try:
        async with session.get(EXTERNAL_API_URL, headers={"Accept-Encoding": "zstd"}) as resp:
            compressed_data = await resp.read()
            encoding = resp.headers.get("Content-Encoding","none")
            print("Content-Encoding",encoding)
            if encoding == "zstd":
                    dctx = zstd.ZstdDecompressor()
                    with dctx.stream_reader(compressed_data) as reader:
                        data = reader.read().decode("utf-8")
            else:
                    data = compressed_data.decode("utf-8")
            print("data :",data)
            
            logger.debug(f"External API response data: {data}")
            logger.info(f"Analysis generated  successfully")

    except Exception as e:
        
        logger.error(f"Error occurred while generating analysis: {e}")

In [30]:
response.usage_metadata

{'input_tokens': 130, 'output_tokens': 671, 'total_tokens': 801}

In [31]:
type(response.usage_metadata)

dict

In [33]:
response.usage_metadata["total_tokens"]

801